In [1]:
%load_ext autoreload
%autoreload 2

# Ingestion

In [4]:
import yfinance as yf
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Float,
    DateTime,
    Integer
)
import sqlite3

In [5]:
def insert_data(db_path: str,
                ticker: str,
                date_start: str,
                date_end: str) -> None:
    try:
        data = yf.download(ticker, start = date_start, end = date_end)
    except Exception as e:
        raise Exception   
    data = data.reset_index().drop("Adj Close", axis=1)
    engine = create_engine(f"sqlite:///{db_path}")
    metadata_obj = MetaData()
    table = Table(
        ticker.lower(),
        metadata_obj,
        Column("id", Integer, primary_key=True),
        Column("date", DateTime),
        Column("open", Float),
        Column("high", Float),
        Column("low", Float),
        Column("close", Float),
        Column("volume", Integer),
    )
    metadata_obj.create_all(engine)
    conn = sqlite3.connect(db_path)
    data.to_sql(
        ticker.lower(),
        conn,
        if_exists='append',
        index=False
    )

In [7]:
insert_data(
    db_path = "../database/stocks.db",
    ticker = "ILMN",
    date_start = "2020-01-01", 
    date_end="2024-12-31"
)

[*********************100%%**********************]  1 of 1 completed


In [9]:
insert_data(
    db_path = "../database/stocks.db",
    ticker = "AAPL",
    date_start = "2020-01-01", 
    date_end="2024-12-31"
)

[*********************100%%**********************]  1 of 1 completed


In [10]:
insert_data(
    db_path = "../database/stocks.db",
    ticker = "NVDA",
    date_start = "2020-01-01", 
    date_end="2024-12-31"
)

[*********************100%%**********************]  1 of 1 completed


Test

In [8]:
import pandas as pd
conn = sqlite3.connect("../database/stocks.db")
pd.read_sql('''SELECT * FROM ilmn LIMIT 5;''', conn)

,id,date,open,high,low,close,volume
0,1,2020-01-02 00:00:00,329.049988,332.850006,323.290009,327.000000,813900
1,2,2020-01-03 00:00:00,322.489990,325.140015,321.010010,322.730011,1346100
2,3,2020-01-06 00:00:00,320.970001,325.709991,318.769989,325.529999,760800
3,4,2020-01-07 00:00:00,328.209991,331.170013,326.109985,329.690002,789100
4,5,2020-01-08 00:00:00,328.320007,334.500000,326.739990,332.160004,679700


# Instantiate agent

In [2]:
from llama_index.core import (
    SQLDatabase,
    Settings,
    VectorStoreIndex
)
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema
)
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.callbacks import CallbackManager,LlamaDebugHandler

import sqlite3
from sqlalchemy import create_engine
import pandas as pd

from IPython.display import display, Markdown

import os
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
Settings.callback_manager = CallbackManager([llama_debug])

LLMs on Bedrock that work include
1. anthropic.claude-3-opus-20240229-v1:0

> Claude-3-sonnet and Mistral Large does not work!

In [4]:
Settings.llm = Bedrock(
    model = "anthropic.claude-3-opus-20240229-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)
Settings.embed_model = HuggingFaceEmbedding(
    model_name = 'jinaai/jina-embeddings-v2-base-en'
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-en
Load pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-en


/app/tlim2/anaconda3/envs/sandbox/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.

INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [5]:
# Get tables
conn = sqlite3.connect("../database/stocks.db")
sql_query = """SELECT name FROM sqlite_master 
WHERE type='table';"""
cursor = conn.cursor()

cursor.execute(sql_query)
tables = cursor.fetchall()

In [6]:
tables = [table[0] for table in tables]
tables

['ilmn', 'aapl', 'nvda']

In [7]:
table_name = {"ilmn": "Illumina",
              "aapl": "Apple",
              "nvda": "Nvidia"}

In [8]:
engine = create_engine("sqlite:///../database/stocks.db")
sql_database = SQLDatabase(engine, include_tables = tables)
table_node_mapping = SQLTableNodeMapping(sql_database)

In [9]:
table_schema_objs = [
    SQLTableSchema(table_name=table,
                   context_str = f"This table gives information regarding the {table_name[table]}'s stock metrics including closing and opening stock prices and volume.")
    for table in tables
]

In [10]:
table_schema_objs

[SQLTableSchema(table_name='ilmn', context_str="This table gives information regarding the Illumina's stock metrics including closing and opening stock prices and volume."),
 SQLTableSchema(table_name='aapl', context_str="This table gives information regarding the Apple's stock metrics including closing and opening stock prices and volume."),
 SQLTableSchema(table_name='nvda', context_str="This table gives information regarding the Nvidia's stock metrics including closing and opening stock prices and volume.")]

## Create index

In [10]:
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 0.408565 seconds
**********


## Create query engine

In [11]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever()
)

In [12]:
query_str = "Describe the general closing price trend for Apple in 2020, 2021, 2022 and 2023. These years were plagued with economic recessions arising from COVID-19. Was Apple heavily affected?"
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'nvda' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Nvidia's stock metrics including closing and opening stock prices and volume.

Table 'aapl' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Apple's stock metrics including closing and opening stock prices and volume.
> Table desc str: Table 'nvda' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Nvidia's stock metrics including closing and opening stock prices and volume.

Table 'aapl'

<b>Based on the closing price data provided for Apple stock from 2020 through 2023, Apple's stock price showed an overall upward trend despite the economic challenges posed by the COVID-19 pandemic during this period.

In early 2020, Apple's stock price hovered around $75-$80. It dipped to around $60 in March 2020 as the pandemic took hold, but then began a steady climb upwards. By late 2020, the stock was trading above $130.

The upward momentum continued in 2021, with the stock reaching the $180 level by year-end, more than doubling from the lows seen in early 2020. 

In 2022, Apple's stock price was more volatile, trading in a range between about $130-$180 for most of the year. However, it ended 2022 near the top of that range around $130.

So far in 2023, Apple's stock has continued to advance, climbing above the $190 level in recent months.

In summary, while there were some dips and volatility, Apple's stock price more than doubled from 2020 to 2023 despite the economic headwinds, demonstrating the company's resilience. The data suggests Apple was not heavily impacted by the pandemic-driven recessions compared to the overall upward trajectory of its stock performance over this multi-year period.</b>

#### Verify

In [15]:
import pandas as pd
conn = sqlite3.connect("../database/stocks.db")
df = pd.read_sql('''SELECT * FROM aapl;''', conn)

In [16]:
df['date'] = pd.to_datetime(df['date'])
df.head()

,id,date,open,high,low,close,volume
0,1,2020-01-02,74.059998,75.150002,73.797501,75.087502,135480400
1,2,2020-01-03,74.287498,75.144997,74.125000,74.357498,146322800
2,3,2020-01-06,73.447502,74.989998,73.187500,74.949997,118387200
3,4,2020-01-07,74.959999,75.224998,74.370003,74.597504,108872000
4,5,2020-01-08,74.290001,76.110001,74.290001,75.797501,132079200


In [17]:
df['year'] = df['date'].dt.year

In [18]:
df[['year', 'close']].groupby('year').mean().reset_index()

,year,close
0,2020,95.347075
1,2021,140.989365
2,2022,154.835060
3,2023,172.549000
4,2024,178.554204


In [19]:
df[['year', 'close']].groupby('year').min().reset_index()

,year,close
0,2020,56.092499
1,2021,116.360001
2,2022,126.040001
3,2023,125.019997
4,2024,165.000000


In [23]:
df[df['year']==2021][['date', 'close']].max()

date     2021-12-31 00:00:00
close             180.330002
dtype: object

# Agent Experiments

In [ ]:
companies = ", ".join(list(table_name.values()))

In [36]:
from llama_index.core.agent import ReActAgent

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.tools.tavily_research.base import TavilyToolSpec

import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

query_engine_tools = [
    QueryEngineTool(
        query_engine = query_engine,
        metadata = ToolMetadata(
            name="stocks",
            description = (
                f"Provides stock information about these companies: '{companies}' including"
                " open, high, low, and closing prices, as well as stock volumes on a"
                " daily basis from 2020-2024. Use a detailed plain text question as"
                " input to the tool."
            )
        )
    )
]

tavily_tool = TavilyToolSpec(
    api_key = os.environ["TAVILY_API_KEY"]
)
query_engine_tools.extend(tavily_tool.to_tool_list())

## ReAct Agent

In [37]:
agent = ReActAgent.from_tools(
    tools = query_engine_tools,
    verbose = True,
    system_prompt = f"""\
    You are a skilled stock analyst designed to analyse stock price trends against
    public information. You are honest. If you do not know the answer, you will
    not make up an answer. You must always use at least one of the tools
    provided when answering a question."""
)

In [38]:
query_string = "Compare and contrast Apple, Nvidia and Illumina's stock price trends from 2020-2024. Which company weathered the economic recession from COVID-19 better? If I have $10,000, which company should I invest in?"
response = agent.chat(query_string)
display(Markdown(f"<b>{response}</b>"))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: stocks
Action Input: {'input': "Compare and contrast Apple, Nvidia and Illumina's stock price trends from 2020-2024. Which company weathered the economic recession from COVID-19 better? If I have $10,000, which company should I invest in?"}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'ilmn' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Illumina's stock metrics including closing and opening stock prices and volume.

Table 'nvda' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Nvidia's stock metrics including closing and opening stock prices and volume.
> Table desc str: Table 'ilmn' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Illumina's stock metrics including closing and opening stock prices and volume.

Table '

<b>Based on the stock price data from 2020-2024, here is a comparison of how Apple, Nvidia and Illumina performed:

Nvidia was the clear winner in terms of stock price appreciation. Its shares soared from around $60 in early 2020 to over $900 by early 2024, a stunning 15x increase. While there was some volatility, especially in late 2022, the overall trajectory was very positive. 

Apple also performed well, although not as spectacularly as Nvidia. Apple's stock reached an all-time high of $197.86 in December 2023 and was trading around $169 as of April 2024. This represents a solid recovery from the COVID-19 downturn and steady gains, although the stock seemed to hit resistance around the $200 level.

Illumina significantly underperformed the other two. Its stock bounced between $300-400 for most of 2020-2021 but then dropped sharply in early 2022 to around $200. While there was some recovery in late 2022 and 2023, the stock ended the period much lower at about $120 in May 2024.

In terms of weathering the initial economic shock from COVID-19 in early 2020, Nvidia's stock recovered the quickest and strongest. Apple also bounced back fairly well</b>

This query is quite complex. The ReAct agent took 3mins to answer

## LATS Agent
This approach takes longer than ReAct because it's searching through the tree. Might not be suitable for SQL data.

In [42]:
from llama_index.packs.agents_lats import LATSPack

import nest_asyncio
nest_asyncio.apply()

In [43]:
lats_agent = LATSPack(
    tools = query_engine_tools,
    llm = Settings.llm
)

In [45]:
query_string = "Compare and contrast Apple, Nvidia and Illumina's stock price trends from 2020-2024. Which company weathered the economic recession from COVID-19 better? If I have $10,000, which company should I invest in?"
response = lats_agent.run(query_string)
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'aapl' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Apple's stock metrics including closing and opening stock prices and volume.

Table 'ilmn' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Illumina's stock metrics including closing and opening stock prices and volume.
> Table desc str: Table 'aapl' has columns: id (INTEGER), date (DATETIME), open (FLOAT), high (FLOAT), low (FLOAT), close (FLOAT), volume (INTEGER), and foreign keys: . The table description is: This table gives information regarding the Apple's stock metrics including closing and opening stock prices and volume.

Table 'ilmn

## Introspective Agent

In [16]:
companies = ", ".join(list(table_name.values()))
companies

'Illumina, Apple, Nvidia'

In [27]:
from llama_index.agent.introspective import (
    ToolInteractiveReflectionAgentWorker,
    IntrospectiveAgentWorker
)
from llama_index.core.agent import FunctionCallingAgentWorker, ReActAgent
from llama_index.core.llms import ChatMessage, MessageRole

In [28]:
def get_introspective_agent_with_tool_interactive_reflection(
    verbose:bool = True, with_main_worker:bool = True
):
    """Helper function for building introspective agent using tool-interactive
    reflection.
    
    Steps:
    1. Define the ToolInteractiveReflectionAgentWorker
        a. Instantiate CritiqueAgentWorker to perform reflection with tools
        b. Define an LLM that will be used to generate corrections against the critique
        c. Define a function that determines the stopping condition for reflection/correction
        d. Construct ToolInteractiveReflectionAgentWorker using from_defaults()
    2. Optionally define a MainAgentWorker
    3. Construct IntrospectiveAgent
        a. Construct IntrospectiveAgentWorker using .from_defaults()
        b. Construct IntrospectiveAgent using .as_agent()
    """
    critique_agent_worker = FunctionCallingAgentWorker.from_tools(
        tools = query_engine_tools, llm = Settings.llm, verbose = verbose
    )
    correction_llm = Settings.llm
    
    def stopping_callable(critique_str: str) -> bool:
        """Function that determines stopping condition for reflection and correction"""
        return "[PASS]" in critique_str
    
    tool_interactive_reflection_agent_worker = (
        ToolInteractiveReflectionAgentWorker.from_defaults(
            critique_agent_worker = critique_agent_worker,
            critique_template = (
                "Retrieve the appropriate records from the SQL database based on the user's question."
                "Check the SQL command and response carefully for correctness, and give constructive criticism on how to improve it."
                "Write '[PASS]' otherwise write '[FAIL]'."
                "Here is the text:\n {input_str}"
            ),
            stopping_callable = stopping_callable,
            correction_llm = correction_llm,
            verbose = verbose
        )
    )
    
    if with_main_worker:
        main_agent_worker = ReActAgent.from_tools(
            tools = query_engine_tools,
            llm = Settings.llm,
            verbose = verbose
        )
    else:
        main_agent_worker = None
    
    introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
        reflective_agent_worker_tool = tool_interactive_reflection_agent_worker,
        main_agent_worker = main_agent_worker,
        verbose = verbose
    )
    
    chat_history = [
        ChatMessage(
            content = f"You are a financial analyst that helps answer questions and provide data insight on historical stock prices of companies. You also augment the information by searching the Internet",
            role = MessageRole.SYSTEM,
        )
    ]
    
    return introspective_agent_worker.as_agent(
        chat_history = chat_history, verbose = verbose
    )

In [29]:
introspective_agent = get_introspective_agent_with_tool_interactive_reflection()

ValueError: Model name anthropic.claude-3-opus-20240229-v1:0 does not support function calling API. 

This approach won't work without OpenAI, and is currently unproven on structured data.